In [1]:
import torch
from model import create_model
from dataloader import SegmentationDataset, get_transform
import utils
import os
import lovasz_losses as L 
import numpy as np

model_path = 'C:/Users/ozhyb/pytorch-licenseplate-segmentation/model.pth'
dataset_dir = './dataset'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_test = SegmentationDataset(os.path.join(dataset_dir, "val"), get_transform(train=False))
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn
)

def evaluate(model, data_loader, device):
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    iou_list = []
    
    with torch.no_grad():
        for image, target in metric_logger.log_every(data_loader, 10, header):
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss, iou = criterion(output, target)
            iou_list.append(iou)
            metric_logger.update(loss=loss.item(), iou=iou, mIOU=np.mean(iou_list))
    
    print(f'{header} mIOU: {np.mean(iou_list)}')
    return np.mean(iou_list)

def criterion(inputs, target):
    sigmoid = torch.sigmoid(inputs['out'])
    preds = (inputs['out'].data > 0).long()
    
    import lovasz_losses as L
    loss = L.lovasz_softmax(sigmoid, target, classes=[1], ignore=128)
    
    if 'aux' in inputs:
        sigmoid_aux = torch.sigmoid(inputs['aux'])
        loss_aux = L.lovasz_softmax(sigmoid_aux, target, classes=[1], ignore=128)
        loss += 0.5 * loss_aux
    
    iou = L.iou_binary(preds, target, ignore=128, per_image=True)
    
    return loss, iou

model = create_model()
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint, strict=False)
model.to(device)

mIOU = evaluate(model, data_loader_test, device)
print(f'mIOU для оригінальної моделі: {mIOU}')


C:\Users\ozhyb\anaconda3\envs\lpsegmentation\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Test:  [  0/201]  eta: 0:10:29  loss: 0.5056 (0.5056)  iou: 0.0000 (0.0000)  mIOU: 0.0000 (0.0000)  time: 3.1336  data: 2.6800  max mem: 924
Test:  [ 10/201]  eta: 0:01:21  loss: 0.5063 (0.5057)  iou: 0.0000 (0.6088)  mIOU: 0.6381 (0.6965)  time: 0.4257  data: 0.2445  max mem: 951
Test:  [ 20/201]  eta: 0:00:50  loss: 0.5063 (0.5059)  iou: 0.0000 (1.0296)  mIOU: 0.5581 (0.6366)  time: 0.1357  data: 0.0011  max mem: 951
Test:  [ 30/201]  eta: 0:00:39  loss: 0.5051 (0.5055)  iou: 0.0000 (0.8745)  mIOU: 0.9036 (0.7452)  time: 0.1249  data: 0.0013  max mem: 1046
Test:  [ 40/201]  eta: 0:00:32  loss: 0.5052 (0.5055)  iou: 0.0000 (0.9512)  mIOU: 0.9750 (0.7992)  time: 0.1224  data: 0.0017  max mem: 1046
Test:  [ 50/201]  eta: 0:00:27  loss: 0.5054 (0.5055)  iou: 0.0000 (0.8192)  mIOU: 0.9069 (0.8101)  time: 0.1109  data: 0.0010  max mem: 1046
Test:  [ 60/201]  eta: 0:00:24  loss: 0.5057 (0.5056)  iou: 0.0000 (0.6892)  mIOU: 0.7982 (0.7995)  time: 0.1162  data: 0.0007  max mem: 1046
Test:  [ 